## Vehicle Re-Identification Training Example

In [1]:
import torchreid
import sys
import os.path as osp
import pandas as pd
from torchreid.data import ImageDataset
import torch
import os

/home/gowna/VehicleReIdentificationDataset/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
2023-11-21 07:43:55.520958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 07:43:55.869765: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
print(f"torchreid versions is: {torchreid.__version__}")
torch.cuda.is_available()

torchreid versions is: 1.4.0


True

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.makedirs("./log", exist_ok=True)


In [4]:
# Some of the available models, which you can directly use with torchreid module
models_list = ['osnet_x1_0' ,'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'resnet50','resnet152']

In [5]:
# class VeRiDataset(ImageDataset):

#     # All you need to do here is to generate three lists,
#     # which are train, query and gallery.
#     # Each list contains tuples of (img_path, pid, camid),
#     # where
#     # - img_path (str): absolute path to an image.
#     # - pid (int): person ID, e.g. 0, 1.
#     # - camid (int): camera ID, e.g. 0, 1.
#     # Note that
#     # - pid and camid should be 0-based.
#     # - query and gallery should share the same pid scope (e.g.
#     #   pid=0 in query refers to the same person as pid=0 in gallery).  --> Very important!
#     # - train, query and gallery share the same camid scope (e.g.
#     #   camid=0 in train refers to the same camera as camid=0
#     #   in query/gallery).

#     def __init__(self, root='', **kwargs):
#         PATH_TO_DATASET = "/data/ml-data/vehicle-reid-dataset/" # This is the folder path where you have extracted the zip file and it contains the gallery, query and train images
#         FOLDER_PATH, DATASET_TYPES, IMAGES = next(os.walk(f"{PATH_TO_DATASET}")) 
#         training_array = []
#         testing_array = []
#         query_array = []
#         DATASET_TYPES = ["image_train","image_gallery","image_query"]
#         for dataset_type in DATASET_TYPES:
#             _, _, IMAGES = next(os.walk(f"{FOLDER_PATH}/{dataset_type}/"))

#             if dataset_type == "image_train" or dataset_type == "image_gallery":

#                 labelSet = []
#                 for idx, image in enumerate(IMAGES):
#                     pid = int(image.split('.jpg')[0].split('_')[-1])
#                     labelSet.append(pid)
#                 label_dict = {label: index for index, label in enumerate(set(labelSet))}
                
#                 for image in IMAGES:
#                     cam_id = int(image.split('.jpg')[0].split('_')[-2])
#                     car_id = label_dict[int(image.split('.jpg')[0].split('_')[-1])]

#                     if dataset_type == "image_gallery":
#                         testing_array.append((FOLDER_PATH +  dataset_type + "/" + image, car_id, cam_id))
#                     else:
#                         training_array.append((FOLDER_PATH +  dataset_type + "/" + image, car_id, cam_id))
                        
#             elif dataset_type == "image_query":
#                 for image in IMAGES:
#                     cam_id = int(image.split('.jpg')[0].split('_')[-2])
#                     car_id = label_dict[int(image.split('.jpg')[0].split('_')[-1])]
#                     query_array.append((FOLDER_PATH +  dataset_type + "/" + image, car_id, cam_id))

            
#         train = training_array
#         query = query_array
#         gallery = testing_array
#         print 

#         super(VeRiDataset, self).__init__(train, query, gallery, **kwargs)

In [6]:
datamanager = torchreid.data.ImageDataManager(
    root="",
    sources="veri",
    height=256,
    width=256,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x256
+ random flip
+ random crop (enlarge to 288x288 and crop 256x256)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x256
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
veri
nbzdjbldjblbbg
<class 'torchreid.data.datasets.image.veri.VeRiDataset'>
600
50
=> Loaded VeRiDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   600 |    50949 |        85
  query    |    50 |      424 |        85
  gallery  |    50 |     3823 |        85
  ----------------------------------------
trainset-----------------   ----------------------------------------
  subset   | # ids | # items | # cameras
  ----------------------------------------
  train    |   600 |   50949 |   

In [9]:
# datamanager = torchreid.data.ImageDataManager(
#     root="/data/ml-data/",
#     sources="market1501",
#     targets="market1501",
#     height=256,
#     width=128,
#     batch_size_train=32,
#     batch_size_test=100,
#     transforms=["random_flip", "random_crop"]
# )

In [11]:
try:
    torchreid.data.register_image_dataset('vehiclereid', VeRiDataset)
    print ("new dataset")
except Exception as e:
    print(e)

'vehiclereid'


In [12]:
LOSS = "SOFTMAX" # Define softmax loss, else triple loss will be usd

In [13]:
for current_model in models_list:
    print(current_model)
    if LOSS == "SOFTMAX":
        model = torchreid.models.build_model(
            name=f'{current_model}',
            num_classes=datamanager.num_train_pids,
            loss='softmax'
        )
    else:
        model = torchreid.models.build_model(
            name=f'{current_model}',
            num_classes=datamanager.num_train_pids,
            loss='triplet'
        )

    model = model.cuda()
    optimizer = torchreid.optim.build_optimizer(
        model,
        optim='adam',
        lr=0.0003
    )

    scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=20
    )

    if LOSS == "SOFTMAX":
        engine = torchreid.engine.ImageSoftmaxEngine(
            datamanager,
            model,
            optimizer=optimizer,
            scheduler=scheduler,
            
        )
    else:
        print("Using ImageTripletEngine")
        engine = torchreid.engine.ImageTripletEngine(
            datamanager,
            model,
            optimizer=optimizer,
            scheduler=scheduler,
        )

    original_stdout = sys.stdout
    with open(f"log/log_{current_model}.txt", 'a') as f:
        sys.stdout = f
        engine.run(
        save_dir=f"log/{current_model}",
        max_epoch=60,
        eval_freq=5,
        print_freq=10,
        test_only=False
        )
        sys.stdout = original_stdout

    del engine
    del model
    del optimizer
    del scheduler

osnet_x1_0
Successfully loaded imagenet pretrained weights from "/home/gowna/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
osnet_ibn_x1_0
Successfully loaded imagenet pretrained weights from "/home/gowna/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
osnet_ain_x1_0


Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /home/gowna/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 35.3MB/s]


Successfully loaded imagenet pretrained weights from "/home/gowna/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
resnet50


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/gowna/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 115MB/s] 


resnet152


Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/gowna/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:02<00:00, 105MB/s]  


In [18]:
!python scripts/main.py \
--config-file configs/im_osnet_x1_0_softmax_256x128_amsgrad.yaml \
--root '' \
model.load_weights log/osnet_x1_0/model/model.pth.tar-60 \
test.evaluate True

/home/gowna/.local/lib/python3.10/site-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
2023-11-20 23:02:50.497673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 23:02:50.734599: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Show configuration
adam:
  beta1: 0.9
  beta2: 0.999
cuhk03:
  classic_split: False
  labeled_images: False
  use_metric_cuhk03: False
data:
  co